# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [90]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config1 import geoapify_key

In [91]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chuy,-33.6971,-53.4616,76.23,54,0,16.46,UY,1675537303
1,1,grindavik,63.8424,-22.4338,36.07,75,75,19.57,IS,1675537343
2,2,georgetown,5.4112,100.3354,76.98,94,20,0.00,MY,1675537343
3,3,ponta do sol,32.6667,-17.1000,63.23,64,5,9.22,PT,1675537344
4,4,geraldton,-28.7667,114.6000,72.01,88,100,9.22,AU,1675537345


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [108]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    geo = True,
    tiles = "OSM"
)

# Display the map plot_1
map_plot_1

# Display the map
# YOUR CODE HERE

c:\Users\user\anaconda3\lib\site-packages\holoviews\util\transform.py:673: RuntimeWarning: invalid value encountered in sqrt
  data = fn(*args, **kwargs)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [124]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_city=city_data_df.loc[(city_data_df["Max Temp"]<=100) & (city_data_df["Max Temp"]>50) &(city_data_df["Wind Speed"]<=4.5)]
ideal_city2= pd.DataFrame(ideal_city)
ideal_city2=ideal_city2.loc[(ideal_city["Cloudiness"]==0)]
ideal_city2

#ideal_city=city_data_df.query(("Max Temp")<26 and ("Wind Speed")<4.5) 
# Drop any rows with null values
# YOUR CODE HERE
#ideal_city2
# Display sample data
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,ushuaia,-54.8000,-68.3000,53.26,50,0,3.44,AR,1675537294
81,81,bredasdorp,-34.5322,20.0403,71.60,83,0,0.72,ZA,1675537354
120,120,russell,32.3502,-85.1999,57.40,33,0,3.44,US,1675537403
144,144,saint george,37.1041,-113.5841,53.44,42,0,3.00,US,1675537204
171,171,richards bay,-28.7830,32.0377,72.54,88,0,2.59,ZA,1675537422
241,241,lompoc,34.6391,-120.4579,64.62,69,0,3.44,US,1675537432
271,271,avanigadda,16.0215,80.9181,69.73,81,0,3.00,IN,1675537461
369,369,lata,40.1629,-8.3327,53.82,74,0,3.94,PT,1675537453
395,395,santa ana,33.7456,-117.8678,71.87,46,0,3.00,US,1675537227
492,492,hyeres,43.1167,6.1167,54.68,71,0,4.50,FR,1675537546


### Step 3: Create a new DataFrame called `hotel_df`.

In [122]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
rad=5000
number=len(ideal_city2)
ideal_city2["Hotel Name"]=""
category="accommodation.hotel"

limit=2
base_url = "https://api.geoapify.com/v2/places"
#print(requests.get(base_url)) 
print("Starting hotel search")
#loop to extract the lat and long and feed it to the JSON request
for index, row in ideal_city2.iterrows():

    lat=row["Lat"]
    long=row["Lng"]
    params={
          "bias": f"proximity:{long},{lat}",        
          "categories":category,
         "limit":limit,
          "filter":f"circle:{long},{lat},{rad}",
          "apiKey":geoapify_key}

    response=requests.get(base_url, params=params).json() #request and capture of data from API
  
    try:
        ideal_city2.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        ideal_city2.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{ideal_city2.loc[index, 'City']} - nearest hotel: {ideal_city2.loc[index, 'Hotel Name']}")

# Display sample data
ideal_city2
     
 


Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
richards bay - nearest hotel: Protea Hotel
santa ana - nearest hotel: Holiday Inn Express


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
81,81,bredasdorp,-34.5322,20.0403,71.60,83,0,0.72,ZA,1675537354,Victoria Hotel
171,171,richards bay,-28.7830,32.0377,72.54,88,0,2.59,ZA,1675537422,Protea Hotel
395,395,santa ana,33.7456,-117.8678,71.87,46,0,3.00,US,1675537227,Holiday Inn Express


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [125]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = ideal_city2.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    geo = True,
    tiles = "ESRI",
    hover_cols = ["Hotel Name"]
   
)

# Display the map plot_1
map_plot_1
#hvplot.help('points')
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)